CS5228 Project - Task 3

Advanced Search Engine Ranking on the Potential Bargain.

Subtasks:
1. Search Engine: Filtering out the cars that satisfying the requirements of buyers.
2. Price Prediction: Predicting the market price cars.
3. Ranking Algorithm: Ranking cars based on the bargain, where bargain = predicted price - listed price.

In [1]:
import csv
from numpy.core.numeric import normalize_axis_tuple
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
from datetime import datetime

from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
import pandas as pd

Subtask 1 - Price Prediction

a. Speed V.S. Memory: Runtime prediction can save memory, but the waiting time will increase obviously when database is larger; If we have enough memory, we can do prediction once a time and save all the results in the database. Here we choose to store market prices in advance.

b. Algorithm: Xgboost

In [2]:
from xgboost import XGBRegressor
from utils.prediction import parse, analyze_attribute, build_vectors, self_split,write_to_csvfile

data_train = parse('data/train.csv')

attrs, attr2vec = analyze_attribute(data_train)


data_train = build_vectors(data_train, attr2vec, attrs)

X = data_train[:, :-1]
y = data_train[:, -1]

test_ratio = 0.3
X_train, X_test, y_train, y_test = self_split(X, y, test_ratio)

other_params ={"n_estimators": 400, "max_depth": 20, "min_child_weight": 2, "colsample_bytree": 0.5}
learning_rate = 0.09
model = XGBRegressor(**other_params, learning_rate=learning_rate)
model.fit(X_train, y_train)


y_pred_test = model.predict(X_test)
write_to_csvfile(y_pred_test, 'data/train.csv', ratio=test_ratio)

(5035,)
(16784, 30)


Subtask 2 - Search Engine

a. Search condition: [make, manufacture (interval value: x-y), transmission, price (interval value: x-y)], where -1 means user do not give any constraint to this condition

b. input: conditions; output: list of satisfying items

In [12]:
from utils.filtering import my_search

filtered = my_search([-1,'2013-2015','manual',-1])
print(filtered.shape)

(96, 32)


Subtask 3 - Ranking Algorithm

a. Ranking condition: bargain is the main factor, listed price is the second factor.

b. input: k, get top-k best

In [13]:
from utils.rank import ranking
ranking(filtered,10)

      Unnamed: 0  listing_id                                title        make  \
79         11828     1000222           Renault Megane RS 265 2.0M     renault   
3618       15367      974592               Land Rover Defender 90  land rover   
1875       13624      890904                         Foton Aumark       foton   
3036       14785     1007974                  Isuzu FXZ77M Tipper       isuzu   
431        12180     1020699                  Isuzu FXZ77M Tipper       isuzu   
426        12175     1013490              Maxus V80 Commuter 2.5M       maxus   
3671       15420     1018078         Mitsubishi Fuso Canter FEB71         NaN   
370        12119     1018087                       Nissan Cabstar      nissan   
4518       16267      995443                          Isuzu TFR86       isuzu   
3888       15637     1014197  Fiat Doblo Cargo Maxi 1.6M Multijet        fiat   

         model                                        description  \
79      megane  upgraded hu, speakers, 